In [73]:
import pandas as pd
import requests
import io
from urllib.parse import urlparse, urljoin, quote
from bs4 import BeautifulSoup
import os
import tqdm

In [74]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

base_url = 'https://transparentcalifornia.com/agencies/salaries'
response = requests.get(base_url, headers=headers)

cities, counties = pd.read_html(io.StringIO(response.text))[:2]
cities = list(cities[0].str.lower().str.replace(' ', '-'))
counties = list(counties[0].str.lower().str.replace(' ', '-'))
munis = cities + counties

years = range(2011, 2023)

urls = [f'https://transparentcalifornia.com/salaries/search/?a={county}&q=election{suffix}&y={year}&page=' for year in years for county in munis for suffix in ['', 's']]

In [84]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

dir = 'transparentcalifornia'
os.makedirs(dir, exist_ok=True)

for url in tqdm.tqdm(urls):
    has_next = True
    page_no = 0
    while has_next:
        page_no += 1
        url_w_page = f"{url}{page_no}"
        response = requests.get(url_w_page, headers=headers)

        df = pd.read_html(io.StringIO(response.text))[0]
        if len(df)>0:
            filename = f"{''.join(urlparse(url_w_page)[2:]).replace('/', '|')}.html"
            with open(f"{dir}/{filename}", 'w') as f:
                f.write(response.text)
            soup = BeautifulSoup(response.text, 'html.parser')
            next_buttons = soup.find_all('li', class_='next')
            has_next = any('disabled' not in li['class'] for li in next_buttons)


  0%|                                                                                                  | 0/12912 [00:00<?, ?it/s]


FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?

In [34]:
response = requests.get('https://transparentcalifornia.com/salaries/download_search/?a=alameda-county&q=elections&y=2022', headers=headers)

In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

url = 'https://transparentcalifornia.com/salaries/download_search/?a=alameda-county&q=elections&y=2022'

# Create a new instance of the Chrome driver
driver = webdriver.Chrome()

try:
    # Navigate to the URL
    driver.get(url)

    # Wait for the "just a moment" message to disappear or for the CSV content to appear
    wait = WebDriverWait(driver, 30)
    wait.until(EC.invisibility_of_element_located((By.XPATH, "//div[contains(text(), 'Just a moment')]")))
    
    # You may need to further inspect the page's HTML to locate the CSV download link or element

    # Once the CSV content is available, retrieve it and process it using Pandas
    csv_data = driver.page_source
    df = pd.read_csv(pd.compat.StringIO(csv_data))
    print(df.head())  # Display the first few rows of the DataFrame

finally:
    # Close the web browser when done
    driver.quit()


AttributeError: module 'pandas.compat' has no attribute 'StringIO'

In [43]:
url = 'https://transparentcalifornia.com/salaries/download_search/?a=alameda-county&q=elections&y=2022'

# Create a new instance of the Chrome driver
driver = webdriver.Chrome()

driver.get(url)